# Metro Region Sports Team Correlations with Win/Loss Ratio

In this project, I read in a file of metropolitan regions and their associated sports teams from an html file to find a few simple statistics. Each region may have had one or more teams from the "Big 4": NFL, MLB, NBA, or NHL, and I used a csv for each league that contained several years of win/loss percentages for each team. All statistics were from the perspective of the metropolitan region, so the html file was the "source of authority" for the location of a given sports team. Thus, teams which are commonly known by a different area (e.g. "Oakland Raiders") needed to be mapped into the metropolitan region given (e.g. San Francisco Bay Area).

In Problem 1, I determined whether a correlation exists between metro region populations and the average of the win percentage for each region's NHL teams in 2018. 

Problem 2 explored the hypothesis that the mean of the winning percentages for all metro areas is equal between two sports by performing paired t-tests for each pair of sports. 

## Problem 1 - NHL Correlation

Below is the code I developed to determine if there is a correlation between NHL team winning percentages and their corresponding metro area population using 2018 data.

### Importing Libraries and Reading In Files

The necessary libraries were imported and the html file read in as 'cities'. The 'Total' row at the bottom of 'cities' was filtered out and I chose the relevant columns I would need from the dataframe. The population column got renamed and changed to numeric form before printing a few columns of the resulting 'cities' dataframe for reference.

In [25]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
cities=pd.read_html("https://raw.githubusercontent.com/slmanner/py-sports/master/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities = cities.rename(columns = {'Population (2016 est.)[8]':'Population'})
cities['Population'] = pd.to_numeric(cities['Population'])
print(cities.iloc[:,[0,1,5]])

         Metropolitan area  Population                             NHL
0            New York City    20153634  RangersIslandersDevils[note 3]
1              Los Angeles    13310447                      KingsDucks
2   San Francisco Bay Area     6657982                  Sharks[note 7]
3                  Chicago     9512999                      Blackhawks
4        Dallas–Fort Worth     7233323                           Stars
5         Washington, D.C.     6131977                        Capitals
6             Philadelphia     6070500                 Flyers[note 13]
7                   Boston     4794447                          Bruins
8   Minneapolis–Saint Paul     3551036                   Wild[note 16]
9                   Denver     2853077              Avalanche[note 18]
10   Miami–Fort Lauderdale     6066387                        Panthers
11                 Phoenix     4661537                         Coyotes
12                 Detroit     4297617                       Red Wings
13    

### Cleaning the 'cities' Dataframe

The 'notes' in the team names were eliminated and I expanded cities with more than one team to multiple rows. The 'citiesNHL' dataframe was printed for reference.

In [26]:
cities['NHL'] = cities['NHL'].str.replace(r'\[note\s\d+]', '', regex = True)
citiesNHL = cities.assign(NHL = cities['NHL'].str.findall(r'[A-Z](?:\s[A-Z]|[^A-Z])+')).explode('NHL').reset_index(drop=True).dropna()
print(citiesNHL.iloc[:,[0,1,5]])

         Metropolitan area  Population             NHL
0            New York City    20153634         Rangers
1            New York City    20153634       Islanders
2            New York City    20153634          Devils
3              Los Angeles    13310447           Kings
4              Los Angeles    13310447           Ducks
5   San Francisco Bay Area     6657982          Sharks
6                  Chicago     9512999      Blackhawks
7        Dallas–Fort Worth     7233323           Stars
8         Washington, D.C.     6131977        Capitals
9             Philadelphia     6070500          Flyers
10                  Boston     4794447          Bruins
11  Minneapolis–Saint Paul     3551036            Wild
12                  Denver     2853077       Avalanche
13   Miami–Fort Lauderdale     6066387        Panthers
14                 Phoenix     4661537         Coyotes
15                 Detroit     4297617       Red Wings
16                 Toronto     5928040     Maple Leafs
19        

### Cleaning the NHL Dataframe and Merging

A function was created for utilization in Problem 2. Within the function, the NHL dataframe was read in and its year filtered to 2018. I printed a list of the teams as a reference of what I was starting with. The first word for each NHL team was eliminated as well as any asterisks at the end. The remaining team names were cleaned up and the 'team' variable was renamed to 'NHL.'

The cities and NHL dataframes were merged and the win percentage for each team calculated with overtime losses included as losses. The teams within each metro region were grouped using the average win percentage for each region. I printed the resulting dataframe for reference.

In [27]:
def nhl_correlation(): 
    nhl_df = pd.read_csv('https://raw.githubusercontent.com/slmanner/py-sports/master/nhl.csv')        
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    print(nhl_df.iloc[:,0])
    nhl_df['team'] = nhl_df['team'].str.replace(r'^\w+\s|\*', '', regex = True)

    nhl_df.replace({'team' : {'Bay Lightning': 'Lightning', 'Jersey Devils': 'Devils', 'Jose Sharks':'Sharks',
                            'York Islanders':'Islanders', 'York Rangers': 'Rangers', 
                          'St. Louis Blues':'Blues','Angeles Kings':'Kings'}}, inplace = True)
    nhl_df.rename(columns = {'team':'NHL'}, inplace = True)
    nhl = citiesNHL.merge(nhl_df,on = 'NHL')
    nhl['L'] = pd.to_numeric(nhl['L'])
    nhl['W'] = pd.to_numeric(nhl['W'])
    nhl['OL'] = pd.to_numeric(nhl['OL'])
    nhl['W%nhl'] = (nhl['W'])/(nhl['L']+nhl['W']+nhl['OL'])
    nhl = nhl.groupby(['Metropolitan area', 'Population'],as_index = False).agg({'W%nhl':np.mean})
    return(nhl)
nhl = nhl_correlation()
print('\n')
print(nhl)

0          Atlantic Division
1       Tampa Bay Lightning*
2             Boston Bruins*
3       Toronto Maple Leafs*
4           Florida Panthers
5          Detroit Red Wings
6         Montreal Canadiens
7            Ottawa Senators
8             Buffalo Sabres
9      Metropolitan Division
10      Washington Capitals*
11      Pittsburgh Penguins*
12      Philadelphia Flyers*
13    Columbus Blue Jackets*
14        New Jersey Devils*
15       Carolina Hurricanes
16        New York Islanders
17          New York Rangers
18          Central Division
19      Nashville Predators*
20            Winnipeg Jets*
21           Minnesota Wild*
22       Colorado Avalanche*
23           St. Louis Blues
24              Dallas Stars
25        Chicago Blackhawks
26          Pacific Division
27     Vegas Golden Knights*
28            Anaheim Ducks*
29          San Jose Sharks*
30        Los Angeles Kings*
31            Calgary Flames
32           Edmonton Oilers
33         Vancouver Canucks
34           A

### Finding the Correlation

Finally, I found the correlation between the metro region population and their NHL teams' mean win percentage and printed the results.

In [28]:
corr, pval = stats.pearsonr(nhl['Population'] , nhl['W%nhl'])
print(corr)

-0.002239945812164773


### Problem 1 - Answer

There is essentially no correlation between the size of a metropolitan area and its NHL win percentage in 2018 because the correlation is very close to 0.

## Problem 2 - Comparing Sports Teams by Metro Region

This problem explored the hypothesis that the mean of the winning percentages for all metro areas is equal between two sports by performing paired t-tests for each pair of sports. To perform this analysis, I performed a series of paired t-tests between each pair of sports. 

Again, values were averaged when a sport had multiple teams in one region. For each sport, I only included cities which have teams engaged in that sport and others were dropped as appropriate.

### Setting Up Problem 2

This problem builds off of problem 1 using the same libraries and the work already performed on the 'cities' dataframe. Similar work was duplicated to the other three major league sports.

In [29]:
# removing 'notes' for remaining three sports in 'cities' dataframe using a loop
sports = ['NBA', 'MLB', 'NFL']
for sport in sports:
    cities[sport].str.split('[A-Z]+', expand=True)
    cities[sport] = cities[sport].str.replace(r'\[note\s\d+]', '', regex = True)       

# expand cities with more than one team to multiple rows for each sport
citiesNBA = cities.assign(NBA = cities['NBA'].str.findall(r'[A-Z,0-9](?:\s[A-Z]|[^A-Z])+')).explode('NBA').reset_index(drop = True).dropna() 
citiesMLB = cities.assign(MLB = cities['MLB'].str.findall(r'[A-Z](?:\s[A-Z]|[^A-Z])+')).explode('MLB').reset_index(drop = True).dropna() 
citiesNFL = cities.assign(NFL = cities['NFL'].str.findall(r'[A-Z,0-9](?:\s[A-Z]|[^A-Z])+')).explode('NFL').reset_index(drop = True).dropna() 

### Cleaning and Preparing csv's for Other Three Sports

The following code is similar to work seen in the 'NHL_correlation' function. (I think I could make the following code more efficient by creating more loops for the remaining three sports and I welcome any suggestions for improvement.)

In [30]:
# Read in csv's for remaining three sports
mlb_df = pd.read_csv('https://raw.githubusercontent.com/slmanner/py-sports/master/mlb.csv')
nba_df = pd.read_csv('https://raw.githubusercontent.com/slmanner/py-sports/master/nba.csv')
nfl_df = pd.read_csv('https://raw.githubusercontent.com/slmanner/py-sports/master/nfl.csv')

# filter year to 2018 for remaining three sports
df_list = [nba_df, mlb_df, nfl_df] 
df_list_2018 = []
for df in df_list:
    df_filter = (df['year'] == 2018)
    df_list_2018.append(df.loc[df_filter])
# clean up the team names in each csv and merge each with the 'cities' dataframe
# then find each metro area win percent
# NBA
nba_df['team'] = df_list_2018[0]['team'].str.replace(r'^\w+\s|\*|\s\(\d+\)', '', regex = True)
nba_df.replace({'team' : {'York Knicks': 'Knicks', 'State Warriors': 'Warriors', 'City Thunder':'Thunder',
                          'Orleans Pelicans':'Pelicans', 'Antonio Spurs': 'Spurs','Angeles Lakers':'Lakers',
                          'Angeles Clippers':'Clippers'}}, inplace = True)
nba_df.rename(columns = {'team':'NBA'}, inplace = True)
nba = citiesNBA.merge(nba_df,on = 'NBA')
nba['L'] = pd.to_numeric(nba['L'])
nba['W'] = pd.to_numeric(nba['W'])
nba['W%nba'] = (nba['W'])/(nba['L']+nba['W'])
# MLB
mlb_df['team'] = df_list_2018[1]['team'].str.replace(r'^\w+\s|\*|\s\(\d+\)', '', regex = True)
mlb_df.replace({'team' : {'York Yankees': 'Yankees', 'Bay Rays': 'Rays', 'City Royals':'Royals', 'Diego Padres': 
                          'Padres','York Mets':'Mets', 'St. Louis Cardinals': 'Cardinals', 'Francisco Giants':'Giants', 
                          'Angeles Angels':'Angels','Angeles Dodgers':'Dodgers'}}, inplace = True)
mlb_df.rename(columns = {'team':'MLB'}, inplace = True)
mlb = citiesMLB.merge(mlb_df,on = 'MLB')
mlb['L'] = pd.to_numeric(mlb['L'])
mlb['W'] = pd.to_numeric(mlb['W'])
mlb['W%mlb'] = (mlb['W'])/(mlb['L']+mlb['W'])
# NFL
nfl_df['team'] = df_list_2018[2]['team'].str.replace(r'^\w+\s|\*|\s\+|\+|\s\(\d+\)', '', regex = True)
nfl_df.replace({'team' : {'England Patriots': 'Patriots', 'City Chiefs': 'Chiefs', 'Bay Packers': 'Packers',
                          'York Jets': 'Jets', 'Orleans Saints': 'Saints', 'York Giants': 'Giants', 'Angeles Rams': 'Rams',
                          'Bay Buccaneers': 'Buccaneers','Angeles Chargers': 'Chargers', 'Francisco 49ers': '49ers'}}, inplace = True)
nfl_df.rename(columns = {'team':'NFL'}, inplace = True)
nfl = citiesNFL.merge(nfl_df,on = 'NFL')
nfl['L'] = pd.to_numeric(nfl['L'])
nfl['W'] = pd.to_numeric(nfl['W'])
nfl['W%nfl'] = (nfl['W'])/(nfl['L']+nfl['W'])
# group the metro areas back together for each sport
nba = nba.groupby(['Metropolitan area'],as_index = False).agg({'W%nba':np.mean})
mlb = mlb.groupby(['Metropolitan area'],as_index = False).agg({'W%mlb':np.mean})
nfl = nfl.groupby(['Metropolitan area'],as_index = False).agg({'W%nfl':np.mean})

### Performing t-tests

The dataframes were merged for each pair of sports, creating six combinations. Paired t-test were performed to compare winning percentages between the sports. (I think I could make this code more efficient by creating a single table with all four sports before performing the t-tests from the single table making sure to drop NA's.)

In [31]:
# create dataframes containing the 6 pairs of sports
nhl_nfl = nhl.merge(nfl, on = 'Metropolitan area')
nhl_nba = nhl.merge(nba, on = 'Metropolitan area')
nhl_mlb = nhl.merge(mlb, on = 'Metropolitan area')    
nfl_nba = nfl.merge(nba, on = 'Metropolitan area')
nfl_mlb = nfl.merge(mlb, on = 'Metropolitan area')    
nba_mlb = nba.merge(mlb, on = 'Metropolitan area')
# perform t-tests on win percentages between pairs of sports
teststat,NHL_NFL = stats.ttest_rel(nhl_nfl['W%nhl'],nhl_nfl['W%nfl'])
teststat,NHL_NBA = stats.ttest_rel(nhl_nba['W%nhl'],nhl_nba['W%nba'])
teststat,NHL_MLB = stats.ttest_rel(nhl_mlb['W%nhl'],nhl_mlb['W%mlb'])
teststat,NFL_NBA = stats.ttest_rel(nfl_nba['W%nfl'],nfl_nba['W%nba'])
teststat,NFL_MLB = stats.ttest_rel(nfl_mlb['W%nfl'],nfl_mlb['W%mlb'])
teststat,NBA_MLB = stats.ttest_rel(nba_mlb['W%nba'],nba_mlb['W%mlb'])

### Printing Results

In [32]:
# create a table of results
p_values = [(np.NaN, NFL_NBA, NHL_NFL, NFL_MLB),(NFL_NBA, np.NaN, NHL_NBA, NBA_MLB), 
    (NHL_NFL, NHL_NBA, np.NaN, NHL_MLB),(NFL_MLB, NBA_MLB, NHL_MLB, np.NaN) ] 
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame(p_values, index = sports, columns =sports) 
# print the results
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.458903,0.802069
NBA,0.941792,NaN,0.973079,0.950540
NHL,0.458903,0.973079,NaN,0.369759
MLB,0.802069,0.950540,0.369759,NaN


## Answer and Final Thoughts

Because the p-values are >0.05 for the t-tests performed on each pair of sports, I failed to reject the hypothesis that the mean of the winning percentages for all metro areas is equal between any two sports. In other words, the results are not statistically significant for any pair of sports. This result makes sense because the expected mean of the winning percentages for each sport is 0.50, but grouping the teams by metro area throws these averages off slightly. What would be more interesting would be to compare averages between pairs of sports for one particular metro region to see if they are statistically different. 